## CHSH SIMULATION VIOLATING THE BELL INEQUALITY


The CHSH game was proposed by Clauser, Horne, Shimony, and Holt demonstrates the violation of Bell's inequality, which fundamentally distinguishes quantum mechanics from classical physics.

WHAT is CHSH Game ?


The CHSH game is a thought experiment that is often used to illustrate the concept of quantum entanglement
The CHSH game is a game of strategy between two players, Alice and Bob. Alice and Bob
are given a set of possible inputs x,y∈{0,1} ,
where Alice produces an output 
𝑎∈{0,1} based on her input x and Bob produces an output b{0,1} 
without communicating with each other.

Win Condition: Alice and Bob win if:
𝑎⊕𝑏=𝑥∧𝑦
where ⊕ is the XOR operation and ∧ is the AND operation.

and they must produce a set of possible outputs. The goal
of the game is to maximize the probability of winning, which is defined as follows

The optimal classical strategy cannot achieve a win probability exceeding 75% over many rounds.
This 75% limit is due to Bell's inequality, which constrains classical correlations.

$
|\Psi⟩ = \frac{1}{\sqrt{2}} (|00⟩ + |11⟩)
$

introducing the Quantum Correlation to acheive the win probability
exceeding the probability of 75%  




In [119]:
import numpy as np 
#classical strategy 
def classical_CHSH():
    win_count=0
    trials=10000

    for _ in range(trials):
        # generate random bits
        x,y=np.random.randint(0,2,2) 
        a,b=0,0
        if (a^b)==(x*y):
            win_count+=1
    
    win_prob=win_count/trials

    return win_prob

# Quantum CHSH Strategy
def quantum_CHSH():
    win_count = 0
    trials = 100000  # Increase trials for better accuracy
    
    # Define measurement angles
    theta_a = [0, np.pi / 4]  # Alice's angles
    theta_b = [np.pi / 8, -np.pi / 8]  # Bob's angles (CORRECTED)
    
    for _ in range(trials):
        # Generate random questions (x, y)
        x, y = np.random.randint(0, 2, 2)
        
        # Select measurement angles based on questions
        angle_a = theta_a[x]
        angle_b = theta_b[y]
        
        # Compute quantum correlation probability for outcome a XOR b = x AND y
        winning_condition = x * y  # x AND y
        prob_win = 0.5 * (1 + (-1)**winning_condition * np.cos(2 * (angle_a - angle_b)))
        
        # Clamp probability to valid range [0, 1]
        prob_win = np.clip(prob_win, 0, 1)
        
        # Simulate the quantum outcome
        if np.random.rand() < prob_win:
            win_count += 1

    # Calculate winning probability
    win_prob = win_count / trials
    return win_prob

# Results
print(f"Classical Winning Probability: {classical_CHSH() * 100:.2f}%")
print(f"Quantum Winning Probability (Corrected Angles): {quantum_CHSH() * 100:.2f}%")


Classical Winning Probability: 74.85%
Quantum Winning Probability (Corrected Angles): 85.39%


In [120]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt



In [159]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

# Function to create the Bell state
def create_bell_state():
    qc = QuantumCircuit(2, 2)  # 2 qubits and 2 classical bits
    qc.h(0)  # Hadamard gate on Alice's qubit
    qc.cx(0, 1)  # CNOT gate to entangle Alice and Bob
    return qc

# Function to apply measurement settings based on the angle
def apply_measurement(qc, x, y):
    # Define the measurement angles based on the strategy
    # Alice's angles (separated by pi/4)
    theta_A0 = 0
    theta_A1 = np.pi / 4

    # Bob's angles (separated by pi/8 and -pi/8)
    theta_B0 = np.pi / 8
    theta_B1 = -np.pi / 8

    # Apply the rotations based on the inputs
    if x == 0 and y == 0:
        qc.ry(-2 * theta_A0, 0)  # Rotation for Alice
        qc.ry(-2 * theta_B0, 1)  # Rotation for Bob
    elif x == 0 and y == 1:
        qc.ry(-2 * theta_A0, 0)  # Rotation for Alice
        qc.ry(-2 * theta_B1, 1)  # Rotation for Bob
    elif x == 1 and y == 0:
        qc.ry(-2 * theta_A1, 0)  # Rotation for Alice
        qc.ry(-2 * theta_B0, 1)  # Rotation for Bob
    elif x == 1 and y == 1:
        qc.ry(-2 * theta_A1, 0)  # Rotation for Alice
        qc.ry(-2 * theta_B1, 1)  # Rotation for Bob

    # Measure the qubits
    qc.measure([0, 1], [0, 1])

# Simulate the CHSH game
def CHSH_game(rounds=1000):
    simulator = AerSimulator()
    wins = 0
    all_counts = []

    for _ in range(rounds):
        # Random inputs x, y
        x = np.random.randint(0, 2)
        y = np.random.randint(0, 2)

        # Create Bell state
        qc = create_bell_state()

        # Apply measurement settings
        apply_measurement(qc, x, y)

        # Visualize the circuit for the first round
        if _ == 0:
            print("Quantum Circuit for the CHSH Game (First Round):")
            print(qc)
            qc.draw(output='mpl')
            plt.show()


        # Execute the circuit
        result = simulator.run(qc, shots=1).result()
        counts = result.get_counts()
        all_counts.append(counts)

        # Get measurement outcome
        outcome = list(counts.keys())[0]  # Extract the measurement result
        a = int(outcome[1])  # Alice's result
        b = int(outcome[0])  # Bob's result

        # Check win condition: a ⊕ b == x ∧ y
        if (a ^ b) == (x & y):
            wins += 1

    # Aggregate counts for visualization
    aggregated_counts = {}
    for counts in all_counts:
        for key, value in counts.items():
            if key in aggregated_counts:
                aggregated_counts[key] += value
            else:
                aggregated_counts[key] = value

    # Plot histogram of results
    if aggregated_counts:  # Ensure there is data to plot
        print("Histogram of Measurement Results:")
        plot_histogram(aggregated_counts)
        plt.show()
    else:
        print("No measurement results to display.")

    # Calculate winning probability
    return wins / rounds

# Run the simulation
quantum_win_prob = CHSH_game(rounds=1000)
print(f"Quantum Winning Probability: {quantum_win_prob * 100:.2f}%")


Quantum Circuit for the CHSH Game (First Round):
     ┌───┐      ┌───────┐ ┌─┐   
q_0: ┤ H ├──■───┤ Ry(0) ├─┤M├───
     └───┘┌─┴─┐┌┴───────┴┐└╥┘┌─┐
q_1: ─────┤ X ├┤ Ry(π/4) ├─╫─┤M├
          └───┘└─────────┘ ║ └╥┘
c: 2/══════════════════════╩══╩═
                           0  1 
Histogram of Measurement Results:
Quantum Winning Probability: 85.00%
